Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [37]:
NAME = ""
COLLABORATORS = ""

---

# Shape Descriptors
In this notebook we will implement the Laplace Beltrami operator and use it to compute the mean curvature and normals as well as the heat kernel signature for an example mesh.

In [38]:
import numpy as np
import openmesh as om
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh
import k3d

ModuleNotFoundError: No module named 'k3d'

In [39]:
mesh = om.read_trimesh("spot_triangulated.obj")

## Laplace-Beltrami
In this task you are going to implement the Laplace-Beltrami operator.
In the lecture we have described this operator as a sum, however in practice it usually makes more sense to write it as a matrix. This matrix can then e.g. be multiplied with the vertex positions to obtain the Discrete Mean Curvature Normal operator.
This matrix can be split into two parts: 
- The mass matrix contains the vertex area
- The (weak) Laplace matrix contains the cotangent weights

In the following tasks you will build these two matrices.
We have implemented two helper functions for you:
- `triangle_area` computes the area of a triangle, given its three vertex positions
- `cotangent_weight` computes the cotangent weight for a given edge. (In the lecture slides you see a factor of 1/2 before the sum, this factor is part of the cotangent weights)

In previous exercises you have seen how to use the functions `diags` and `csr_matrix` to construct sparse matrices.\
In the introduction we introduced you to openmesh and have shown you how to obtain vertex coordinates, face-vertex indices and iterate over vertices or a vertex neigbourhood. Similarly you can iterate over faces, with `mesh.faces()` or obtain the faces, neighbouring a vertex with `mesh.vf(v)`
The id of a face is obtained from a facee handle with `idx()` (similarly as for vertices).\
Furthermore, for this exercise you will need to get the two vertices of an edge. 
These can be obtained with `mesh.to_vertex_handle(mesh.halfedge_handle(eh,0))`. Replace 0 with 1 to obtain the other vertex.

In [40]:
# helper function: given triangle vertices, compute area
def triangle_area(a,b,c):
    return np.linalg.norm(np.cross(b-a,c-a))/2
    
def mass_matrix(mesh):
    ### YOUR CODE HERE

    M = np.zeros(mesh.n_vertices())
    
    points = mesh.points()

    for face in mesh.faces():
        i_fvs = mesh.fv(face)
        v1 = next(i_fvs).idx()
        v2 = next(i_fvs).idx()
        v3 = next(i_fvs).idx()
        m = triangle_area(points[v1], points[v2], points[v3]) / 3
        M[v1] += m
        M[v2] += m
        M[v3] += m

    return diags(M)
    

# helper function: given edge, compute cotangent weight
def cotangent_weight(mesh, eh):
    v0 = mesh.point(mesh.to_vertex_handle(mesh.halfedge_handle(eh,0)))
    v1 = mesh.point(mesh.to_vertex_handle(mesh.halfedge_handle(eh,1)))
    weight = 0
    for i in range(2):
        heh = mesh.halfedge_handle(eh,i)
        opp_v = mesh.point(mesh.to_vertex_handle(mesh.next_halfedge_handle(heh)))
        dir1 = opp_v - v0
        dir2 = opp_v - v1
        weight += np.dot(dir1,dir2) / np.linalg.norm(np.cross(dir1,dir2))
    return weight / 2

def laplace_matrix(mesh):
    rows, cols, data = [], [], []

    for vertex in mesh.vertices():
        sum_weights = 0
        for he in mesh.ve(vertex):
            
            v0 = mesh.to_vertex_handle(mesh.halfedge_handle(he,0)).idx()
            v1 = mesh.to_vertex_handle(mesh.halfedge_handle(he,1)).idx()
            
            if(v0 != vertex.idx()):
                v1 = v0
                v0 = vertex.idx()

            w = cotangent_weight(mesh, he)
            rows.append(v0)
            cols.append(v1)
            data.append(w)
            
            sum_weights -= w

        rows.append(vertex.idx())
        cols.append(vertex.idx())
        data.append(sum_weights)

    return csr_matrix((data, (rows, cols)))
            

def invert_diagonal_matrix(D):
    d = D.diagonal()
    zero_mask = d == 0
    return diags(np.where(zero_mask, 1, 1/d))



In [41]:
M = mass_matrix(mesh)
M_inv = invert_diagonal_matrix(M)
L = laplace_matrix(mesh)

hn = M_inv.dot(L.dot(mesh.points()))
h = np.linalg.norm(hn, axis=1)
normals = hn / h[:,None]

In the following plot you should see curved areas colored in yellow and flat areas colored in purple.
Furthermore the normals should be orthogonal to the surface and point outwards in convex areas.

In [ ]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=h/2, 
                 color_map=k3d.colormaps.matplotlib_color_maps.viridis,
                color_range=(0,10))
plot += k3d.vectors(mesh.points(), normals/10, line_width=0.0001, use_head=False)
plot

In [42]:
np.testing.assert_array_almost_equal(M.todense()[range(5),range(5)], 
                                     [[0.00478942, 0.00284357, 0.00094711, 0.0006429, 0.00267439]], decimal=8)

In [43]:
np.testing.assert_approx_equal(L.todense()[0,0], -4.230992097869423, significant=8)
np.testing.assert_approx_equal(L.todense()[0,812], 1.4068019986623206, significant=8)
np.testing.assert_approx_equal(L.todense()[729,2923], 1.0595635160548063, significant=8)

## Heat Kernel Signature
In this task you will use the previously defined Laplace-Beltrami operator to compute the Heat Kernel Signature.
You can compute the eigenvalues and eigenvectors using scipy with the function  call `eigsh(L, n_eig, M, sigma=0)`.
Besides the previously computed (weak) Laplacian and mass matrix, the function `heat_kernel_signature(L, M, n_eig, t)` gets as input the number of eigenvalues we will use, and the time step.
You can check your result by trying out several values for the time step

In [44]:
def heat_kernel_signature(L, M, n_eig, t):
    ### YOUR CODE HERE
    eigen_values, eigen_vectors = eigsh(L, n_eig, M, sigma=0)
    k = np.sum(np.exp(eigen_values * t) * (eigen_vectors * eigen_vectors), axis=1)
    return k

In [45]:
vals = heat_kernel_signature(L,M,100,1e-3)

In [ ]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=vals, 
                 color_map=k3d.colormaps.matplotlib_color_maps.viridis)
plot

In [46]:
vals = heat_kernel_signature(L,M,100,1e-3)
np.testing.assert_approx_equal(vals.min(), 9.59263418573553, significant=8)
np.testing.assert_approx_equal(vals.max(), 30.681926901103605, significant=8)